# Turbopump Design Cycle Analysis

Date: 22/12/2024

Author: Elias Aoubala



In [1]:
import numpy as np
import handcalcs.render
import pandas as pd
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP

## 1 - Background

This document aims to ouline the steady state cycle analysis for the turbopump propulsion unit as a function of multiple key parameters.

The main objectives of this study is to figure out the following:

- How does the delivery conditions vary as a function of GG inlet pressures.
- How does shaft speed change as a function of inlet pressures.

## 2 - Method

A simplisitic set of model functions will be created to model the turbopumps behaviour based on a set of nominal delivery powers.

In [2]:
class Turbine():
    
    def __init__(self,
                 N: float):
        
        self._N = N
        

In [4]:
class GasGenerator():
    
    def __init__(self,
                 m_dot: float,
                 o_f: float,
                 P_c: float,
                 Oxid: str = "Nitrous",
                 Fuel: str = "IPA"):
        
        self._m_dot = m_dot
        self._o_f = o_f
        self._P_c = P_c
        self._oxid = Oxid
        self._Fuel = Fuel
        
        return
    
    def evaluate_geometry(self, 
                          P_f: float, 
                          P_o: float,
                          c_star: float
                          ) -> None:
        """
        Function that solves for the injector geometry of the gas generator
        This equatino assumes the SPI model

        Args:
            P_f (float): Nominal Fuel Supply Pressure
            P_o (float): Nominal Oxidiser Supply Pressure
            c_star (float): C* efficiency
        """
        
        # Fuel Orifice
        
        self._rho_f = 1
        
        dp_f = P_f - self._P_c
        
        m_dot_f = self._m_dot/(1 + self._o_f)
        
        self._CdAf = m_dot_f/(2 * self._rho_f * dp_f)**(1/2)
        
        # Oxidiser Orifice
        
        self._rho_o = 1
        
        dp_o = P_o - self._P_c
        
        m_dot_o = self._m_dot - m_dot_f
        
        self._CdAo = m_dot_o/(2 * self._rho_o * dp_o)**(1/2)
        
        # Combustion Chamber
        
        T_c = 1
        
        gamma = 1
        
        rho_c = 1
        
        param = gamma * rho_c * self._P_c * (2/(gamma + 1))**((gamma + 1)/(gamma - 1))
        
        self._CdAc = self._m_dot/(param)**(1/2)
        
        return
    
    def injector_flow(self,
                      P_d: float,
                      P_c: float,
                      prop: str
                      ) -> float:
        """This equation solves for the mass flow rate through a given injector

        Args:
            P_d (float): Injector Delivery Pressure
            P_c (float): Combustion Chamber Pressure

        Returns:
            float: Mass Flow rate through injector
        """
        
        if prop == "Oxidiser":
            cd_a = self._CdAo
            rho_a = self._rho_o
            
        elif prop == "Fuel":
            cd_a = self._CdAf
            rho_a = self._rho_f
            
        m_dot = cd_a * (2 * rho_a * (P_d - P_c))**(1/2)
        
        return m_dot
    
    
    def error_func(self: float,
                P_c_g: float,
                P_f: float,
                P_o: float,
                c_star: float,
                ) -> float:
    
        # We evaluate for our injector flow rates
    
        m_dot_f = self.injector_flow(P_d = P_f,
                                     P_c = P_c_g,
                                     prop = "Fuel")
        
        m_dot_o = self.injector_flow(P_d = P_o,
                                     P_c = P_c_g,
                                     prop = "Oxidiser")
        
        m_dot_inj = m_dot_o + m_dot_f
        
        # We now evaluate for the combustion chamber cnoditions
        
        o_f = m_dot_o/m_dot_f
        
            
    
    def performance_evaluation(self,
                               P_f: float,
                               P_o: float,
                               c_star: float,
                               ) -> list[float]:
        """
        This equation solves for the updated performance conditions of the gas generator given a change of inlet pressures.
        This process
        
        This is inherintly an iterative process to figure out what the new chamber pressure and flow rates of the gas generator will be, hence we use an auxilary function which we can then use the scipy library to figure out what the requried mass flow rates are.

        Args:
            P_f (float): Fuel Delivery Pressure
            P_o (float): Oxidiser Delivery Pressure
            c_star (float): C* efficiency

        Returns:
            list[float]: 
        """
            
        
        
        
        
        
        
        
        